This simple script is used to validate if new versions of FOS don't lead to performance degradations. It uses the training of a simple convolutional network as a reference. On a NVidia GTX 1080TI should take about 17 seconds.

In [1]:
# Uncomment the following line if running on Google Colab
# !pip install fos

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# Import the Fos classes we are going to use in this notebook
from fos import Workout

In [2]:
model = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3),
    nn.ReLU(),
    nn.BatchNorm2d(num_features=16),
    nn.MaxPool2d(4),

    nn.Conv2d(16, 64, kernel_size=3),
    nn.ReLU(),
    nn.Conv2d(64, 64, kernel_size=3),
    nn.ReLU(),
    nn.BatchNorm2d(num_features=64),
    nn.MaxPool2d(4),

    nn.Conv2d(64, 256, kernel_size=3),
    nn.ReLU(),
    nn.Conv2d(256, 256, kernel_size=3),
    nn.ReLU(),
    nn.BatchNorm2d(num_features=256),
    nn.MaxPool2d(4),

    nn.Flatten(),
    nn.Linear(1024, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
    nn.ReLU()
).to("cuda")



In [3]:
# Will speedup a few %
# model = torch.jit.trace(model, torch.rand(16, 3, 224, 224).to("cuda"))
# model = torch.jit.script(model)

In [4]:
data = [(np.float32(np.random.randn(16, 3, 224, 224)), np.float32(np.random.randn(16, 10))) for _ in range(100)]


In [5]:
workout = Workout(model, F.mse_loss)
workout.fit(data)

[  1:   100] - loss: 0.9419 - time: 1.8s


In [6]:
%%time
workout.fit(data, epochs=10)

[  2:   200] - loss: 0.9405 - time: 1.8s
[  3:   300] - loss: 0.9217 - time: 1.8s
[  4:   400] - loss: 0.9280 - time: 1.8s
[  5:   500] - loss: 0.9181 - time: 1.8s
[  6:   600] - loss: 0.8951 - time: 1.8s
[  7:   700] - loss: 0.9298 - time: 1.8s
[  8:   800] - loss: 0.9146 - time: 1.8s
[  9:   900] - loss: 0.8863 - time: 1.8s
[ 10:  1000] - loss: 0.8890 - time: 1.8s
[ 11:  1100] - loss: 0.8654 - time: 1.8s
CPU times: user 17.9 s, sys: 32 ms, total: 17.9 s
Wall time: 17.9 s
